In [1]:
# !sudo apt-get install libtinfo5
# !python -m venv myenv
# !pip install torch==2.1.0 torchvision --upgrade

'sudo'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


     ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
     -------- ------------------------------- 1.2/5.7 MB 37.0 MB/s eta 0:00:01
     ---------------------- ----------------- 3.2/5.7 MB 40.6 MB/s eta 0:00:01
     ------------------------------------ --- 5.2/5.7 MB 48.0 MB/s eta 0:00:01
     ---------------------------------------  5.7/5.7 MB 45.8 MB/s eta 0:00:01
     ---------------------------------------- 5.7/5.7 MB 36.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/536.2 kB ? eta -:--:--
     ------------------------------------- 536.2/536.2 kB 17.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/192.3 MB ? eta -:--:--
   ---------------------------------------- 2.1/192.3 MB 66.4 MB/s eta 0:00:03
    --------------------------------------- 4.1/192.3 MB 64.5 MB/s eta 0:00:03
   - -------------------------------------- 6.0/192.3 MB 64.2 MB/s eta 0:00:03
   - -------------------------------------- 7.8/192.3 MB 55.6 MB/s eta 0:

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# transform = ToTensor()를 사용해서 데이터셋을 텐서로 변환시킬 수 있습니다.
# 오픈 데이터셋에서 학습 데이터를 다운로드하세요.
training_data = datasets.FashionMNIST(
    root="/tmp/data",
    train=True, # 학습 데이터를 지정
    download=True,
    transform=ToTensor(), # 텐서로 변환
)

# 오픈 데이터셋에서 테스트 데이터를 다운로드하세요.
test_data = datasets.FashionMNIST(
    root="/tmp/data",
    train=False, # 테스트 데이터를 지정
    download=True,
    transform=ToTensor(), # 텐서로 변환
)

100%|██████████| 26421880/26421880 [00:06<00:00, 4068800.30it/s]


Extracting /tmp/data\FashionMNIST\raw\train-images-idx3-ubyte.gz to /tmp/data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 105766.64it/s]


Extracting /tmp/data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to /tmp/data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:02<00:00, 1580747.04it/s]


Extracting /tmp/data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to /tmp/data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<?, ?it/s]

Extracting /tmp/data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to /tmp/data\FashionMNIST\raw


In [4]:
# 미니 배치 학습
batch_size = 64

# 데이터 로더 만들기
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


#### 모델 만들기

In [5]:
# PyTorch에서 인공신경망을 정의하기 위해 nn.Module을 상속하는 클래스를 생성
# 훈련에 사용할 cpu, gpu나 mps device 가져오기
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 모델 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


#### 모델 파라미터 최적화

In [6]:
loss_fn = nn.CrossEntropyLoss()     # 손실함수로 크로스엔트로피를 사용한다.
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)  # 경사하강법으로 최적화된 파라미터를 찾으며, 학습률은 10의 -3승으로 설정한다.

In [7]:
# 단일 train 순환에서 모델은 train 데이터셋에 대해 예측하고 예측 오류를 역전파(Back propaganda)를 이용해서 모델의 파라미터를 조정
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오차 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
# 테스트 데이터셋에 대한 모델의 성능을 확인하여 학습이 제대로 이루어지고 있는지 확인
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309915  [   64/60000]
loss: 2.288436  [ 6464/60000]
loss: 2.274080  [12864/60000]
loss: 2.267854  [19264/60000]
loss: 2.235236  [25664/60000]
loss: 2.222524  [32064/60000]
loss: 2.236331  [38464/60000]
loss: 2.205964  [44864/60000]
loss: 2.196496  [51264/60000]
loss: 2.158480  [57664/60000]
Test Error: 
 Accuracy: 40.7%, Avg loss: 2.157734 

Epoch 2
-------------------------------
loss: 2.174232  [   64/60000]
loss: 2.153783  [ 6464/60000]
loss: 2.105566  [12864/60000]
loss: 2.122394  [19264/60000]
loss: 2.050462  [25664/60000]
loss: 2.005322  [32064/60000]
loss: 2.045302  [38464/60000]
loss: 1.968817  [44864/60000]
loss: 1.970545  [51264/60000]
loss: 1.887103  [57664/60000]
Test Error: 
 Accuracy: 49.7%, Avg loss: 1.892033 

Epoch 3
-------------------------------
loss: 1.930401  [   64/60000]
loss: 1.887758  [ 6464/60000]
loss: 1.783833  [12864/60000]
loss: 1.829454  [19264/60000]
loss: 1.697480  [25664/60000]
loss: 1.656397  [32064/600

#### 모델 저장하기

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


#### 모델 불러오기
1. 모델 구조를 다시 만들어서
2. 그 안의 내부 상태 사전을 불러옴

In [11]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

#### 모델을 통한 예측하기

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
